# Generate circlized plot for schizophrenia analysis DEG results

In [ ]:
library(biomaRt)
library(circlize)
library(tidyverse)
library(ComplexHeatmap)

## Prepare data

### Get gene annotation

In [ ]:
ensembl = useEnsembl(biomart="ensembl", dataset="hsapiens_gene_ensembl")
biomart = getBM(attributes=c('ensembl_gene_id', 'chromosome_name', 
                             'start_position', 'end_position'), mart=ensembl)
biomart %>% head(2)

### Get logFC for differential expression analysis

In [ ]:
deg = data.table::fread("../../_m/genes/diffExpr_szVctl_full.txt") %>%
    select(gencodeID, ensemblID, Symbol, logFC, "adj.P.Val")
deg %>% head(2)

### Merge data and select regions

In [ ]:
bed = deg %>% 
    inner_join(biomart, by=c("ensemblID"="ensembl_gene_id")) %>%
    select(chromosome_name, start_position, end_position, logFC, "adj.P.Val") %>%
    mutate(chromosome_name=paste0('chr', chromosome_name))

bed_sz = bed %>% filter(logFC > 0, adj.P.Val < 0.05)
bed_ctl = bed %>% filter(logFC < 0, adj.P.Val < 0.05)
bed_nonsig = bed %>% filter(adj.P.Val > 0.05)

## Circos Plot

In [ ]:
plot_circos <- function(bed_list, colors, tissue_name){
    lgd_points = Legend(at=c("Upregulated in SZ", "Downregulated in SZ"), 
                        type="points", legend_gp=gpar(col = c("red", "blue")), 
                        title_position="topleft", title="DE Analysis", 
                        background="#FFFFFF")
    circos.clear() # clear plot if there is any
    circos.par("start.degree" = 0) # rotate 90 degrees
    # initialize with ideogram
    # use hg38, default is hg19
    circos.initializeWithIdeogram(species="hg38") 
    # add logFC as a genomic track
    circos.genomicTrack(bed_list, track.height=inches_h(1),
                        text(0, 0, tissue_name, cex = 1.5),
                        panel.fun = function(region, value, ...) {
                            i = getI(...)
                            circos.genomicPoints(region, value, pch = 16, 
                                                 cex = 0.6, col = colors[i], ...)
    })
    draw(lgd_points, x=unit(5, "mm"), y=unit(5, "mm"), just=c("left", "bottom"))
}

In [ ]:
tissue = 'Caudate Nucleus'
tissue_name = gsub(" ", "_", tissue)

In [ ]:
plot_circos(list(bed_sz, bed_ctl), c("red", "blue"), tissue)

In [ ]:
png(filename = paste0("significant_circos_plot_", tissue_name, ".png"))
plot_circos(list(bed_sz, bed_ctl), c("red", "blue"), tissue)
dev.off()

pdf(file = paste0("significant_circos_plot_", tissue_name, ".pdf"))
plot_circos(list(bed_sz, bed_ctl), c("red", "blue"), tissue)
dev.off()

svg(filename = paste0("significant_circos_plot_", tissue_name, ".svg"))
plot_circos(list(bed_sz, bed_ctl), c("red", "blue"), tissue)
dev.off()

In [ ]:
plot_circos(list(bed_nonsig, bed_sz, bed_ctl), 
            c("gray", "red", "blue"), tissue)

In [ ]:
png(filename = paste0("allGenes_circos_plot_", tissue_name, ".png"))
plot_circos(list(bed_nonsig, bed_sz, bed_ctl), 
            c('gray', 'red', 'blue'), tissue)
dev.off()

pdf(file = paste0("allGenes_circos_plot_", tissue_name, ".pdf"))
plot_circos(list(bed_nonsig, bed_sz, bed_ctl), 
            c('gray', 'red', 'blue'), tissue)
dev.off()

svg(filename = paste0("allGenes_circos_plot_", tissue_name, ".svg"))
plot_circos(list(bed_nonsig, bed_sz, bed_ctl), 
            c('gray', 'red', 'blue'), tissue)
dev.off()

## Repreducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()